In [ ]:
!pip install yfinance pandas scikit-learn matplotlib tensorflow
!pip install yfinance pandas scikit-learn matplotlib streamlit pyngrok
! pip install streamlit -q

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import pickle



In [ ]:
# Apple stock (AAPL)
data = yf.download("AAPL", start="2018-01-01", end="2024-01-01")
data.head()


/tmp/ipython-input-3095497354.py:2: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download("AAPL", start="2018-01-01", end="2024-01-01")
[*********************100%***********************]  1 of 1 completed


Price,Close,High,Low,Open,Volume
Ticker,AAPL,AAPL,AAPL,AAPL,AAPL
Date,,,,,
2018-01-02,40.381001,40.390379,39.677744,39.888723,102223600
2018-01-03,40.373974,40.917828,40.310683,40.444300,118071600
2018-01-04,40.561497,40.664642,40.338800,40.446630,89738400
2018-01-05,41.023293,41.110026,40.566177,40.657600,94640000
2018-01-08,40.870934,41.166300,40.772474,40.870934,82271200


In [ ]:
data['Target'] = (data['Close'].shift(-1) > data['Close']).astype(int)
data = data.dropna()


In [ ]:
# Features: yesterday’s Close + Volume
X = data[['Close', 'Volume']].shift(1).dropna()
y = data['Target'][1:]  # align target

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, shuffle=False, test_size=0.2
)

# Train model
model = LogisticRegression()
model.fit(X_train, y_train)

# Predictions
preds = model.predict(X_test)

# Evaluation
print("Accuracy:", accuracy_score(y_test, preds))
print(confusion_matrix(y_test, preds))
print(classification_report(y_test, preds))


Accuracy: 0.4966887417218543
[[ 91  49]
 [103  59]]
              precision    recall  f1-score   support

           0       0.47      0.65      0.54       140
           1       0.55      0.36      0.44       162

    accuracy                           0.50       302
   macro avg       0.51      0.51      0.49       302
weighted avg       0.51      0.50      0.49       302



In [ ]:

# Use only Close price for LSTM
scaler = MinMaxScaler()
scaled = scaler.fit_transform(data[['Close']])

# Create sequences
def create_sequences(data, window=30):
    X, y = [], []
    for i in range(len(data)-window-1):
        X.append(data[i:i+window])
        y.append(1 if data[i+window] > data[i+window-1] else 0)
    return np.array(X), np.array(y)

X_seq, y_seq = create_sequences(scaled, window=30)
X_train, X_test = X_seq[:-200], X_seq[-200:]
y_train, y_test = y_seq[:-200], y_seq[-200:]

# LSTM model
model_lstm = Sequential([
    LSTM(50, return_sequences=False, input_shape=(30,1)),
    Dense(1, activation="sigmoid")
])

model_lstm.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
model_lstm.fit(X_train, y_train, epochs=5, batch_size=16, validation_split=0.1)

# Evaluate
loss, acc = model_lstm.evaluate(X_test, y_test)
print("LSTM Accuracy:", acc)


/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/5
72/72 ━━━━━━━━━━━━━━━━━━━━ 8s 38ms/step - accuracy: 0.5375 - loss: 0.6932 - val_accuracy: 0.5156 - val_loss: 0.6925
Epoch 2/5
72/72 ━━━━━━━━━━━━━━━━━━━━ 4s 25ms/step - accuracy: 0.5105 - loss: 0.6939 - val_accuracy: 0.5156 - val_loss: 0.6935
Epoch 3/5
72/72 ━━━━━━━━━━━━━━━━━━━━ 3s 28ms/step - accuracy: 0.5434 - loss: 0.6897 - val_accuracy: 0.5156 - val_loss: 0.6922
Epoch 4/5
72/72 ━━━━━━━━━━━━━━━━━━━━ 3s 38ms/step - accuracy: 0.5495 - loss: 0.6883 - val_accuracy: 0.4844 - val_loss: 0.6935
Epoch 5/5
72/72 ━━━━━━━━━━━━━━━━━━━━ 4s 26ms/step - accuracy: 0.5237 - loss: 0.6908 - val_accuracy: 0.5156 - val_loss: 0.6920
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.5469 - loss: 0.6910  
LSTM Accuracy: 0.550000011920929


In [ ]:
# Download stock data (example: AAPL)
data = yf.download("AAPL", start="2018-01-01", end="2024-01-01")
data['Target'] = (data['Close'].shift(-1) > data['Close']).astype(int)
data = data.dropna()

# Features
X = data[['Close', 'Volume']].shift(1).dropna()
y = data['Target'][1:]  # align target

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=False, test_size=0.2)

# Train Logistic Regression
model = LogisticRegression()
model.fit(X_train, y_train)

# Save model
with open("stock_model.pkl", "wb") as f:
    pickle.dump(model, f)

print("✅ Model trained and saved as stock_model.pkl")


/tmp/ipython-input-2519827228.py:2: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download("AAPL", start="2018-01-01", end="2024-01-01")
[*********************100%***********************]  1 of 1 completed

✅ Model trained and saved as stock_model.pkl


In [ ]:
!wget -q -O - ipv4.icanhazip.com

34.106.7.216


In [ ]:
! streamlit run app.py & npx localtunnel --port 8501



⠙⠹⠸⠼⠴
  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.106.7.216:8501

⠦⠧⠇⠏⠋your url is: https://dirty-aliens-roll.loca.lt
/content/app.py:18: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, period="5d", interval="1d")
[*********************100%***********************]  1 of 1 completed
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/_array_api.py:839: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  array = numpy.asarray(array, order=order, dtype=dtype)
  Stopping...
^C
